In [1]:
import os
import pandas as pd
import numpy as np
import sys

# from konlpy.tag import Kkma

# sys.path.append('..')

# from Code.config import *
# from Code.preprocessingSentence import *

### 문제 List
* preprocessingentence 함수는 input sentence의 길이가 일정 수준 이상인 경우, null 값 return
    
    ```python 
    preprocessingSentence(df['content'].iloc[9], kkma)
    ```

In [4]:
path = '../Data/Raw/탈모톡톡_19~23세탈모.csv'
df = pd.read_csv(path)

In [5]:
df.head(2)

,title,category,date,writer,rank,view,href,content,imageList
0,22인데 이게 맞냐,[22세탈모],08월 08일,모자람있는사람,4,조회 409,https://daedamo.com/1923club/126082?overlaps=1,<ㅜㅠㅜ>곧있으면 전역하는 짬찌인데 말출나와서 오랜만에 정수리 사진 찍으니깐 입대 ...,['https://daedamo.com/new/data/file/1923club/6...
1,20살 탈모,[20세탈모],08월 13일,K29614872442308…,4,조회 212,https://daedamo.com/1923club/126108?overlaps=1,재수하면서 이렇게 됐는데 다시 회복 가능한가요ㅠㅠ 스트레스받아서 너무 힘들어요\r지...,['https://daedamo.com/new/data/file/1923club/4...


In [ ]:
kkma = Kkma()
result = df.iloc[:10].apply(lambda x : preprocessingSentence(x['title'] + x['content'], kkma), axis = 1)

### 오늘의 할 일
* multi-processing
    * multi-processing 함수 파라미터 제공
    * multi-processing 함수 jupyter notebook 실행
* save 함수 생성
* 감정분석 :  preprocessingSentence.py: sentimentAnalysis 선언
* LDA 
    * 주요기능
        * TF-IDF는 적정 크기의 키워드 수 선택이 필요한 경우 사용(https://wikidocs.net/40710)
        * LDA 실행 함수(https://wikidocs.net/30708)
            * input
                * Corpus:
                * Num_topics:
                * Id2word:
                * Passes:
            * output
        * LDA 시각화 함수
            * pyLDAvis.gensim_models
            * parameters
                * ldaModel: 
                * Corpus:
                * Dictionary: 
        * LDA 주요 키워드 도출 함수
        * 토픽별 추세 분석 함수
        * 토픽별 Pain-Point 분석
    * 작동 특징
        * ldaAnalysis
        * document별 토픽 category 저장
        * config.py 생성: parameter 관리
* generalAnalysis
    * 주요기능
        * 전체 분석
            * 전체 비율 
            * 전체 Trend(시계열)
                * 게시글 수
                * 작성자 수
        * 특정 column별 분석
            * column별 비율
            * column별 Trend(시계열)
                * 게시글 수
                * 작성자 수

In [2]:
def change(listValue):
    try:
        return eval(listValue)
    except:
        print(listValue)
        return np.nan

In [3]:
df = pd.read_excel('../Data/Preprocessed/탈모톡톡_19~23세탈모.xlsx')

# df = df[df.columns[:13]]
# df['content'] = df['content'].str.replace('\n', ' ')
# df['title'] = df['title'].str.replace('\n', ' ')
# df['titleContent'] = df['titleContent'].str.replace('\n', ' ')
# df['preprocessedSentence'] = df['preprocessedSentence'].str.replace('\n', ' ')
df['keyword'] = df['keyword'].apply(lambda x : change(x))
df = df[~pd.isnull(df['keyword'])]

df.to_excel('../Data/Preprocessed/탈모톡톡_19~23세탈모.xlsx', index = False)

In [4]:
df = pd.read_excel('../Data/Preprocessed/샴푸&두피케어_샴푸&영양제.xlsx')
df['keyword'] = df['keyword'].apply(lambda x : change(x))
df = df[~pd.isnull(df['keyword'])]
# df = df[df.columns[:13]]
# df['content'] = df['content'].str.replace('\n', ' ')
# df['title'] = df['title'].str.replace('\n', ' ')
# df['titleContent'] = df['titleContent'].str.replace('\n', ' ')
# df['preprocessedSentence'] = df['preprocessedSentence'].str.replace('\n', ' ')



df.to_excel('../Data/Preprocessed/샴푸&두피케어_샴푸&영양제.xlsx', index = False)

In [44]:
df['keyword']

0       ['알다', '신', '후기', '알', '피신', '샴푸', '알다', '탈모',...
1       ['모발', '이식', '후', '탈모', '샴푸', '사용', '관련', '사항'...
2       ['알다', '신', '더블', '이펙트', '샴푸', '유통', '기한', '샴푸...
3       ['비오틴', '거가', '좋', '비오틴', '먹다', '하다', '거가', '좋...
4       ['내일', '모발', '이식', '받다', '영양', '제도', '챙기다', '먹...
                              ...                        
4805    ['포', '샴푸', '주문', '포', '그다', '이태리', '만들다', '임상...
4806    ['샴푸', '효과', '있다', '후기', '사이트', '제휴', '가져오다', ...
4807    ['샴푸', '추천', '부탁', '이때', '머리', '관리', '하다', '이젠...
4808    ['존', '스킨', '샴푸', '후기', '존', '스킨', '샴푸', '정도',...
4809    ['계면', '활성제', '질문', '샴푸', '발모', '효과', '기대', '없...
Name: keyword, Length: 4810, dtype: object

In [7]:
from gensim import corpora

dictionary = corpora.Dictionary(df['keyword'])

In [8]:
dictionary

In [9]:
corpus = [dictionary.doc2bow(text) for text in df['keyword']]

In [13]:
len(corpus)

4809

In [19]:
import pyLDAvis
import pyLDAvis.gensim_models as pyGensim
import pickle

pyLDAvis.enable_notebook()

with open(file = '../Result/LDA/탈모톡톡_19~23세탈모/__15.pickle', mode = 'rb') as f:
    lda = pickle.load(f)
    ldaModel = lda[0]
    dictionary = lda[1]
    corpus = lda[2]

    vis = pyGensim.prepare(ldaModel, corpus, dictionary)

In [20]:
pyLDAvis.display(vis)

In [21]:
ldaModel.print_topics(num_words = 20)

[(0,
  '0.123*"미" + 0.071*"옥" + 0.040*"바르다" + 0.032*"녹" + 0.029*"노" + 0.024*"성" + 0.023*"차" + 0.021*"기능" + 0.017*"테" + 0.015*"발르다" + 0.015*"발라보다" + 0.014*"부산" + 0.013*"맥주" + 0.012*"효모" + 0.012*"올라오다" + 0.012*"반삭" + 0.010*"언제쯤" + 0.010*"써보다" + 0.010*"직구" + 0.008*"은"'),
 (1,
  '0.073*"미녹시딜" + 0.048*"바르다" + 0.046*"쓰다" + 0.036*"하다" + 0.035*"샴푸" + 0.027*"있다" + 0.021*"좋" + 0.019*"효과" + 0.016*"되다" + 0.013*"같" + 0.011*"발르다" + 0.010*"사용" + 0.010*"추천" + 0.009*"사람" + 0.009*"보다" + 0.009*"하루" + 0.008*"크" + 0.008*"글" + 0.007*"뿌리다" + 0.007*"어떻"'),
 (2,
  '0.118*"먹다" + 0.089*"약" + 0.030*"복용" + 0.030*"시" + 0.028*"프로" + 0.026*"되다" + 0.024*"있다" + 0.024*"하다" + 0.022*"같" + 0.021*"효과" + 0.017*"테" + 0.017*"달" + 0.016*"아보" + 0.015*"부작용" + 0.014*"없" + 0.013*"다트" + 0.012*"정도" + 0.010*"사람" + 0.009*"카피" + 0.009*"보다"'),
 (3,
  '0.033*"하다" + 0.025*"녹" + 0.024*"먹다" + 0.024*"실" + 0.023*"술" + 0.023*"운동" + 0.017*"마이" + 0.017*"담배" + 0.017*"생활" + 0.014*"마" + 0.013*"좋" + 0.012*"습관" + 0.008*"끊다" + 0.008*"식습관" + 0.008*"마시다"